# DINEOF results

In this script we are going to visualise the results of the DINEOF reconstruction

In [ ]:
using NCDatasets
using PyPlot
using Missings
using Dates
using Statistics
include("dineof_scripts.jl")

In [ ]:
#read the initial (cloudy) data, land-sea mask, time, latitude and longitude
ds = Dataset("sst_L3_Adriatic.nc");
tmp = ds["SST"][:];
sst  = nomissing(tmp,NaN);


mask = nomissing(ds["mask"][:]);

time = nomissing(ds["time"][:]);
lat = nomissing(ds["lat"][:]);
lon = nomissing(ds["lon"][:]);
close(ds)

#read the reconstructed file
ds = Dataset("sst_L4_dineof_Adriatic.nc");
tmp = ds["sst_filled"][:];
sstf  = nomissing(tmp,NaN);
close(ds)
@show size(sst);
sstf[sstf.==9999].=NaN;

In [ ]:
#Change the variable time to the date 
mydate = Date(2017,1,1) .+ Dates.Day.(time);
extrema(lat)

In [ ]:
#Plot initial data and reconstruction side-by-side to compare
#set index i to one of the time steps of the data matrices (multiple plots)
#i=14
ca = (11,17)
for i = 30
    figure()
subplot(2,2,1)
contourf(lon,lat,mask',levels = [0., 0.5],colors =[[.5,.5,.5]])
pcolor(lon,lat,sst[:,:,i]',cmap="RdYlBu_r"),clim(ca),colorbar(orientation="horizontal")
#title(DateTime(2017,1,1) + Dates.Millisecond(time[i] * 1000*24*60*60))
title(mydate[i])

ylim(39,46)
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);

subplot(2,2,2)
pcolor(lon,lat,sstf[:,:,i]',cmap="RdYlBu_r"),clim(ca),colorbar(orientation="horizontal")
contourf(lon,lat,mask',levels = [0., 0.5],colors =[[.5,.5,.5]]);
ylim(39,46)
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);
title("DINEOF analysis")
end

In [ ]:
# Histogram of SST data (we aim at a similar distribution in the initial and reconstructed datasets, 
# but with a larger sample size in the reconstructed one).

hist(sstf[:],50,label= "DINEOF");
hist(sst[:],50,label="Initial data");
ylabel("Number of data")
xlabel("℃")
legend();

In [ ]:
# Difference between the whole domain average value for each dataset
# Initial data are noisier because the average is done over less data, 
# normally heterogeneously distributed

sst1 = nanmean(sstf,1);
sst2 = nanmean(sst1,2);

sst1i = nanmean(sst,1);
sst2i = nanmean(sst1i,2);

#sst2 = squeeze(sst2);
@show size(sst2)
plot(mydate,sst2i[:],color="r",label="Initial data");
plot(mydate,sst2[:],label="DINEOF");
xlabel("year day");
ylabel("℃     ",rotation=0);
legend();

In [ ]:
#Another example of reconstructed data
for i = 90
    figure()
pcolor(lon,lat,sst[:,:,i]',cmap="RdYlBu_r"),ColorMap(gray);clim(19,27);colorbar() #clim(21,27);
contourf(lon,lat,copy(mask'),levels = [0., 0.5],colors =[[.5,.5,.5]]);
ylim(39,46)
    title("$i")
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);
      
    figure()
pcolor(lon,lat,sstf[:,:,i]',cmap="RdYlBu_r"),ColorMap(gray);clim(19,27);colorbar() #clim(21,27);
contourf(lon,lat,copy(mask'),levels = [0., 0.5],colors =[[.5,.5,.5]]);
ylim(39,46)
    title("$i")
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);
    
    
end

Let's look now at the EOF modes that have been used to reconstruct these data

In [ ]:
#First the Spatial and temporal EOF modes and the singular values X = USV'

ds = Dataset("eof.nc");
tmp = ds["Usst"][:];
Usst  = nomissing(tmp,NaN);
Usst[Usst.==9999].=NaN;

tmp = ds["V"][:];
V  = nomissing(tmp,NaN);

tmp = ds["Sigma"][:];
Sigma  = nomissing(tmp,NaN);

@show size(Usst)
@show size(V)
@show size(Sigma);

In [ ]:
#We can also read how mush of the total variability each mode explains
ds = Dataset("DINEOF_diagnostics.nc");
tmp = ds["varEx"][:];
varEx  = nomissing(tmp,NaN);
varEx

In [ ]:
# The spatial EOFs (Usst) have the size of the initial data (in space) times the number of modes retained for validation
# The temporal EOFs size is the temporal dimension times the number of retained EOFs
# The singular values have the size of the retained EOFs

In [ ]:
plot(Sigma);

In [ ]:
eofn = 1;

subplot(2,1,1)
contourf(lon,lat,mask',levels = [0., 0.5],colors =[[.5,.5,.5]])
pcolor(lon,lat,-Usst[:,:,eofn]',cmap="RdYlBu_r"),colorbar()
ylim(40,46)
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);

title("EOF mode $eofn")
subplot(2,1,2)
plot(mydate,-V[:,eofn]);
grid("on")
xlabel("year day");
@show size(lon)
@show size(lat)
@show size(Usst); #You can plot as many EOFs as the third dimension of this variable
#plotattr("size")